In [ ]:
import numpy as np
from dataset import get_images, get_labels, url, file_names
from convolution import Convolution
from pooling import Pooling
from affine import Affine
from relu import Relu
from softmaxWithLoss import SoftmaxWithLoss
from dropout import Dropout
from model import Model
from optimizer import SGD, Momentum, AdaGrad, RMSprop, Adam
from trainer import Trainer
from util import scale
from metrics import Factory

In [ ]:
#データセット読み込み
x_train = get_images(url, file_names[0])
t_train = get_labels(url, file_names[1], one_hot=None)
x_test = get_images(url, file_names[2])
t_test = get_labels(url, file_names[3], one_hot=None)
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

In [ ]:
def check_shape(x, layers):
    print(x.shape)
    for layer in layers:
        x = layer.forward(x)
        print(type(layer), x.shape)

In [ ]:
#モデル生成
m = Model()
m.append(Convolution(scale(1*3*3) * np.random.randn(8, 1, 3, 3), np.zeros(8)))
m.append(Relu())
m.append(Convolution(scale(8*3*3) * np.random.randn(8, 8, 3, 3), np.zeros(8)))
m.append(Relu())
m.append(Pooling())
#m.append(Convolution(scale(8*3*3) * np.random.randn(8, 8, 3, 3), np.zeros(8)))
#m.append(Relu())
#m.append(Convolution(scale(8*3*3) * np.random.randn(8, 8, 3, 3), np.zeros(8)))
#m.append(Relu())
#m.append(Pooling())
#m.append(Affine(scale(8*4*4) * np.random.randn(8*4*4, 50), np.zeros(50)))
m.append(Affine(scale(8*12*12) * np.random.randn(8*12*12, 50), np.zeros(50)))
m.append(Relu())
#m.append(Dropout())
m.append(Affine(scale(50) * np.random.randn(50, 10), np.zeros(10)))
#m.append(Dropout())
m.append_loss(SoftmaxWithLoss())
m.summary()

In [ ]:
#学習
#check_shape(x_train[:10], m.layers)
t = Trainer(m, SGD(lr=0.1), Factory.create(Factory.MTYPE.MTR, lbl=[0,1,2,3,4,5,6,7,8,9]))
t.fit(x_train, t_train, 10, 100)

In [ ]:
#テスト
met = Factory.create(Factory.MTYPE.MTR, lbl=[0,1,2,3,4,5,6,7,8,9])
ret = np.array([])
for i in range(len(x_test)):
    score = m.predict(x_test[[i]])
    ret = np.append(ret, met.evaluate(t_test[[i]], score))
ret = np.sum(ret.reshape(-1, 4), axis=0) / len(x_test)
print('acc %.4f | f1 %.4f' %(ret[0], ret[3]))